# Trading API

#### 0. Imports

In [51]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import requests
from dotenv import load_dotenv
import os
from typing import Optional, List, Dict, Union
from datetime import datetime, timedelta, timezone
from bson.objectid import ObjectId
import numpy as np

load_dotenv()
API_KEY = os.environ.get("ALPACA_KEY")
ALPACA_SECRET = os.environ.get("ALPACA_SECRET")
ALPACA_AUTH = os.environ.get("ALPACA_AUTH")

headers = {
    "accept": "application/json",
    "content-type": "application/json",
    "authorization": ALPACA_AUTH
}

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 1. Introduction to this notebook

This notebook as a working file to develop the functions that interact with Alpaca's API and MongoDB to set up and monitor trades

# 2. Trading workflow

In [120]:
clients_account_id = {"client_1": "b5284860-b133-458b-8140-8fb5e59d8439"}

In [121]:
def get_account_trading_details(account_id):
    url = f"https://broker-api.sandbox.alpaca.markets/v1/trading/accounts/{account_id}/account"

    response = requests.get(url, headers=headers)

    print(response.json())

    return response.json()

Make INCOMING (flowing into the wallet) simulated bank transaction for the client into his account wallet. 
1. Get the client-bank relationship_id.

In [122]:
account_id = clients_account_id["client_1"]

url = f"https://broker-api.sandbox.alpaca.markets/v1/accounts/{account_id}/ach_relationships"

response = requests.get(url, headers=headers)

print(response.json())
relationship_id = response.json()[0]["id"]
print(f"The relationship_id is {relationship_id}")

[{'id': 'f4a5cd49-7fae-4389-91ae-3ef55d3e144b', 'account_id': 'b5284860-b133-458b-8140-8fb5e59d8439', 'created_at': '2025-02-07T14:11:38.385935Z', 'updated_at': '2025-02-07T14:18:51.678877Z', 'status': 'APPROVED', 'account_owner_name': 'Vibrant Nightingale', 'bank_account_type': 'CHECKING', 'bank_account_number': '32131231abc', 'bank_routing_number': '123103716', 'nickname': 'Bank of America Checking', 'processor_token': None}]
The relationship_id is f4a5cd49-7fae-4389-91ae-3ef55d3e144b


In [123]:
clients_account_id["client_1"]

'b5284860-b133-458b-8140-8fb5e59d8439'

In [124]:
account_id = clients_account_id["client_1"]

def get_account_relationship_id(account_id: str):
    url = f"https://broker-api.sandbox.alpaca.markets/v1/accounts/{account_id}/ach_relationships"

    response = requests.get(url, headers=headers)

    relationship_id = response.json()[0]["id"]
    return relationship_id


relationship_id = get_account_relationship_id(clients_account_id["client_1"])

print(f"The relationship_id for client '{clients_account_id['client_1']}' is '{relationship_id}'.")

The relationship_id for client 'b5284860-b133-458b-8140-8fb5e59d8439' is 'f4a5cd49-7fae-4389-91ae-3ef55d3e144b'.


2. Make the transaction (max. 1 per day)

In [57]:
url = f"https://broker-api.sandbox.alpaca.markets/v1/accounts/{account_id}/transfers"

payload = {
    "transfer_type": "ach",
    "direction": "INCOMING",
    "timing": "immediate",
    "relationship_id": f"{relationship_id}",
    "amount": "10"
}
headers = {
    "accept": "application/json",
    "content-type": "application/json",
    "authorization": ALPACA_AUTH
}

response = requests.post(url, json=payload, headers=headers)

print(response.json())



{'id': 'a2cfac0f-23ff-4a7d-8915-bd7505752d80', 'relationship_id': 'f4a5cd49-7fae-4389-91ae-3ef55d3e144b', 'account_id': 'b5284860-b133-458b-8140-8fb5e59d8439', 'type': 'ach', 'status': 'QUEUED', 'currency': 'USD', 'amount': '10', 'instant_amount': '0', 'direction': 'INCOMING', 'created_at': '2025-02-07T11:20:00.072828645-05:00', 'updated_at': '2025-02-07T11:20:00.072828645-05:00', 'expires_at': '2025-02-14T11:20:00.072214555-05:00', 'reason': None, 'hold_until': None, 'requested_amount': '10', 'fee': '0', 'fee_payment_method': 'user'}


In [125]:
def make_funding_transaction(account_id: str, relationship_id, amount):
    url = f"https://broker-api.sandbox.alpaca.markets/v1/accounts/{account_id}/transfers"

    payload = {
        "transfer_type": "ach",
        "direction": "INCOMING",
        "timing": "immediate",
        "relationship_id": f"{relationship_id}",
        "amount": str(amount)
    }

    response = requests.post(url, json=payload, headers=headers)

    print(response.json())

    return response.json()

make_funding_transaction(clients_account_id['client_1'], relationship_id, 50000)

{'code': 40010001, 'message': 'maximum total daily transfer allowed is $50000'}


{'code': 40010001, 'message': 'maximum total daily transfer allowed is $50000'}

In [59]:
def get_open_positions_client(account_id: str, symbol_id: Optional[str] = None):
    url = f"https://broker-api.sandbox.alpaca.markets/v1/trading/accounts/{account_id}/positions"

    if symbol_id:
        url += f"/{symbol_id}"

    response = requests.get(url, headers=headers)

    print(response.json())

    return response.json()

In [60]:
def close_all_open_positions_client(account_id):
    url = f"https://broker-api.sandbox.alpaca.markets/v1/trading/accounts/{account_id}/positions"

    response = requests.delete(url, headers=headers)

    print(response.json())

    return response.json()

In [61]:
def close_symbol_position_client(account_id: str, symbol: str, magnitude: float, mode: str="percentage"):
    url = f"https://broker-api.sandbox.alpaca.markets/v1/trading/accounts/{account_id}/positions/{symbol}"

    if mode == "percentage":
        url += f"?percentage={magnitude}"
    elif mode == "qty":
        url += f"?qty={magnitude}"
    else:
        raise ValueError("The mode especified is incorrect. It should be either 'percentage' or 'qty'")

    response = requests.delete(url, headers=headers)

    print(response.json())

    return response.json()

In [62]:
def list_open_positions_by_client_id(account_id):
    url = f"https://broker-api.sandbox.alpaca.markets/v1/trading/accounts/{account_id}/positions"

    response = requests.get(url, headers=headers)

    print(response.json())

    return response.json()

In [63]:
def get_order_by_id(account_id, order_id):
    url = f"https://broker-api.sandbox.alpaca.markets/v1/trading/accounts/{account_id}/orders/{order_id}"

    response = requests.get(url, headers=headers)

    print(response.json())

    return response.json()

In [64]:
def get_list_of_orders(
    account_id: str,
    status: Optional[str] = "all",
    limit: Optional[int] = None,
    after: Optional[Union[str, datetime]] = None,
    until: Optional[Union[str, datetime]] = None,
    direction: Optional[str] = None,
    nested: Optional[bool] = None,
    symbols: Optional[str] = None,
    qty_below: Optional[Union[str, float]] = None,
    qty_above: Optional[Union[str, float]] = None,
) -> Dict:
    """
    Retrieve a list of orders for a specific account with optional filters.

    Args:
        account_id (str): The ID of the account for which to retrieve orders.
        status (str, optional): Order status to be queried. Can be 'open', 'closed', or 'all'. Defaults to 'open'.
        limit (int, optional): The maximum number of orders in the response. Defaults to 50, max is 500.
        after (Union[str, datetime], optional): A timestamp in ISO 8601 format or a datetime object. 
            The response will include only orders submitted after this timestamp (exclusive).
        until (Union[str, datetime], optional): A timestamp in ISO 8601 format or a datetime object. 
            The response will include only orders submitted until this timestamp (exclusive).
        direction (str, optional): The chronological order of the response based on submission time. 
            Can be 'asc' or 'desc'. Defaults to 'desc'.
        nested (bool, optional): If True, the result will roll up multi-leg orders under the 'legs' field of the primary order.
        symbols (str, optional): A comma-separated list of symbols to filter by (e.g., 'AAPL,TSLA').
        qty_below (Union[str, float], optional): Filter orders with a quantity below this value.
        qty_above (Union[str, float], optional): Filter orders with a quantity above this value.

    Returns:
        dict: A JSON response containing the list of orders matching the specified filters.
    """
    
    # Function implementation here
    pass
    # Base URL for the Alpaca Broker API
    url = f"https://broker-api.sandbox.alpaca.markets/v1/trading/accounts/{account_id}/orders?"

    # Add status if provided
    if status:
        url += f"&status={status}"

    # Add limit if provided
    if limit:
        url += f"&limit={limit}"

    # Add after timestamp if provided
    if after:
        url += f"&after={after}"

    # Add until timestamp if provided
    if until:
        url += f"&until={until}"

    # Add direction if provided
    if direction:
        url += f"&direction={direction}"

    # Add nested flag if provided
    if nested:
        url += f"&nested={nested}"

    # Add symbols if provided (comma-separated list)
    if symbols:
        url += f"&symbols={symbols}"

    # Add qty_below if provided
    if qty_below:
        url += f"&qty_below={qty_below}"

    # Add qty_above if provided
    if qty_above:
        url += f"&qty_above={qty_above}"


    response = requests.get(url, headers=headers)

    return response.json()

orders = get_list_of_orders(
    account_id="your_account_id",
    status="open",
    limit=100,
    after="2021-01-01T18:38:00",
    until="2021-03-01T18:38:00",
    direction="desc",
    nested=True,
    symbols="AAPL,TSLA",
    qty_below="80",
    qty_above="100"
)
print(orders)

{'code': 40010001, 'message': 'invalid account id'}


In [65]:
def cancel_all_open_orders(account_id):
    url = f"https://broker-api.sandbox.alpaca.markets/v1/trading/accounts/{account_id}/orders"

    response = requests.delete(url, headers=headers)

    print(response.json())

In [66]:
def cancel_open_order(account_id, order_id):
    url = f"https://broker-api.sandbox.alpaca.markets/v1/trading/accounts/{account_id}/orders/{order_id}"

    response = requests.delete(url, headers=headers)

    if response.status_code == 204:
        print(f"Order '{order_id}' cancelled succesfully.")
    else:
        print("Request error.")


In [67]:
def create_order_for_client(
    account_id: str,
    symbol: str,
    qty: Optional[Union[str, float]] = None,
    notional: Optional[Union[str, float]] = None,
    side: str = "buy",
    type: str = "market",
    time_in_force: str = "day",
    limit_price: Optional[Union[str, float]] = None,
    stop_price: Optional[Union[str, float]] = None,
    trail_price: Optional[Union[str, float]] = None,
    trail_percent: Optional[Union[str, float]] = None,
    extended_hours: bool = False,
    client_order_id: Optional[str] = None,
    order_class: str = "simple",
    tags: Optional[List[str]] = None,
    take_profit: Optional[Dict] = None,
    stop_loss: Optional[Dict] = None,
    commission: Optional[float] = None,
    commission_type: str = "notional",
    source: Optional[str] = None,
    instructions: Optional[str] = None,
    subtag: Optional[str] = None,
    swap_fee_bps: Optional[Union[str, float]] = None,
    position_intent: Optional[str] = None,
) -> Dict:
    """
    Create an order for a client's account.

    Args:
        account_id (str): The ID of the account for which to create the order.
        symbol (str): Symbol or asset ID to identify the asset to trade.
        qty (str, optional): Number of shares to trade. Can be fractional for market and day orders.
        notional (str, optional): Dollar amount to trade. Cannot be used with qty.
        side (str, optional): Represents the side of the transaction. Defaults to "buy".
        type (str, optional): The order type. Defaults to "market". Equity assets have the following options: limit, stop, stop_limit, trailing_stop.
        time_in_force (str, optional): The time the order will last without being fulfilled until cancellation. Defaults to "day".
        limit_price (str, optional): Required if type is 'limit' or 'stop_limit'.
        stop_price (str, optional): Required if type is 'stop' or 'stop_limit'.
        trail_price (str, optional): Required if type is 'trailing_stop' and trail_percent is not provided.
        trail_percent (str, optional): Required if type is 'trailing_stop' and trail_price is not provided.
        extended_hours (bool, optional): If true, order is eligible to execute in pre/post-market. Defaults to False.
        client_order_id (str, optional): A unique identifier for the order. Automatically generated if not provided.
        order_class (str, optional): The order class. Defaults to "simple".
        tags (list, optional): List of order tags (max 4).
        take_profit (dict, optional): Take profit order details.
        stop_loss (dict, optional): Stop loss order details.
        commission (float, optional): The commission to collect from the user.
        commission_type (str, optional): How to interpret the commission value. Defaults to "notional".
        source (str, optional): Source of the order.
        instructions (str, optional): Additional instructions for the order.
        subtag (str, optional): Subtag for the order.
        swap_fee_bps (str, optional): Swap fee in basis points.
        position_intent (str, optional): Represents the desired position strategy.

    Returns:
        dict: A JSON response containing the details of the created order.
    """
    url = f"https://broker-api.sandbox.alpaca.markets/v1/trading/accounts/{account_id}/orders"


    payload = {
        "symbol": symbol,
        "qty": qty,
        "notional": notional,
        "side": side,
        "type": type,
        "time_in_force": time_in_force,
        "limit_price": limit_price,
        "stop_price": stop_price,
        "trail_price": trail_price,
        "trail_percent": trail_percent,
        "extended_hours": extended_hours,
        "client_order_id": client_order_id,
        "order_class": order_class,
        "tags": tags,
        "take_profit": take_profit,
        "stop_loss": stop_loss,
        "commission": commission,
        "commission_type": commission_type,
        "source": source,
        "instructions": instructions,
        "subtag": subtag,
        "swap_fee_bps": swap_fee_bps,
        "position_intent": position_intent,
    }

    # Remove None values from payload
    payload = {k: v for k, v in payload.items() if v is not None}

    response = requests.post(url, json=payload, headers=headers)

    if response.status_code == 200:
        print(f"Succesfully placed order {response.json()['id']} for {response.json()['position_intent']} of the symbol {response.json()['symbol']}. See details in response.")

    return response.json()


# order_response = create_order_for_client(
#     account_id="client_account_id",
#     symbol="AAPL",
#     qty="4.122",
#     side="buy",
#     type="market",
#     time_in_force="day",
#     commission=1.0,
#     commission_type="notional",
#     take_profit =  { "limit_price": "3.14" },
#     stop_loss =  {
#         "limit_price": "3",
#         "stop_price": "3"
#     }
# )
# print(order_response)

# Make trades

Invest 100 dolars in Microsoft stocks. We need to trade with "notional" argument of `create_order_for_client()`.

In [68]:
order_created_response = create_order_for_client(account_id=clients_account_id["client_1"], symbol="MSFT", notional="100", side="buy",type="market")
order_created_response

Succesfully placed order 5af6850d-5fdb-4bfb-8455-7a1700e81c78 for buy_to_open of the symbol MSFT. See details in response.


{'id': '5af6850d-5fdb-4bfb-8455-7a1700e81c78',
 'client_order_id': '631fe556-2e1d-458f-858a-72cb3bf84e4c',
 'created_at': '2025-02-07T16:20:01.636996446Z',
 'updated_at': '2025-02-07T16:20:01.638425316Z',
 'submitted_at': '2025-02-07T16:20:01.636996446Z',
 'filled_at': None,
 'expired_at': None,
 'canceled_at': None,
 'failed_at': None,
 'replaced_at': None,
 'replaced_by': None,
 'replaces': None,
 'asset_id': 'b6d1aa75-5c9c-4353-a305-9e2caa1925ab',
 'symbol': 'MSFT',
 'asset_class': 'us_equity',
 'notional': '100',
 'qty': None,
 'filled_qty': '0',
 'filled_avg_price': None,
 'order_class': 'simple',
 'order_type': 'market',
 'type': 'market',
 'side': 'buy',
 'position_intent': 'buy_to_open',
 'time_in_force': 'day',
 'limit_price': None,
 'stop_price': None,
 'status': 'pending_new',
 'extended_hours': False,
 'legs': None,
 'trail_percent': None,
 'trail_price': None,
 'hwm': None,
 'commission': '0',
 'commission_type': 'notional',
 'subtag': None,
 'source': None,
 'expires_at':

In [69]:
get_list_of_orders(clients_account_id["client_1"])

[{'id': '5af6850d-5fdb-4bfb-8455-7a1700e81c78',
  'client_order_id': '631fe556-2e1d-458f-858a-72cb3bf84e4c',
  'created_at': '2025-02-07T16:20:01.636996446Z',
  'updated_at': '2025-02-07T16:20:01.645085066Z',
  'submitted_at': '2025-02-07T16:20:01.636996446Z',
  'filled_at': '2025-02-07T16:20:01.644232458Z',
  'expired_at': None,
  'canceled_at': None,
  'failed_at': None,
  'replaced_at': None,
  'replaced_by': None,
  'replaces': None,
  'asset_id': 'b6d1aa75-5c9c-4353-a305-9e2caa1925ab',
  'symbol': 'MSFT',
  'asset_class': 'us_equity',
  'notional': '100',
  'qty': None,
  'filled_qty': '0.242780909',
  'filled_avg_price': '411.894',
  'order_class': 'simple',
  'order_type': 'market',
  'type': 'market',
  'side': 'buy',
  'position_intent': 'buy_to_open',
  'time_in_force': 'day',
  'limit_price': None,
  'stop_price': None,
  'status': 'filled',
  'extended_hours': False,
  'legs': None,
  'trail_percent': None,
  'trail_price': None,
  'hwm': None,
  'commission': '0',
  'commi

In [70]:
list_open_positions_by_client_id(clients_account_id["client_1"])

[{'asset_id': 'b6d1aa75-5c9c-4353-a305-9e2caa1925ab', 'symbol': 'MSFT', 'exchange': 'NASDAQ', 'asset_class': 'us_equity', 'asset_marginable': True, 'qty': '0.963086428', 'avg_entry_price': '415.331363', 'side': 'long', 'market_value': '396.753085', 'cost_basis': '399.999999', 'unrealized_pl': '-3.246914', 'unrealized_plpc': '-0.0081172850202932', 'unrealized_intraday_pl': '-3.246914', 'unrealized_intraday_plpc': '-0.0081172850202932', 'current_price': '411.96', 'lastday_price': '415.82', 'change_today': '-0.0092828627771632', 'qty_available': '0.963086428'}]


[{'asset_id': 'b6d1aa75-5c9c-4353-a305-9e2caa1925ab',
  'symbol': 'MSFT',
  'exchange': 'NASDAQ',
  'asset_class': 'us_equity',
  'asset_marginable': True,
  'qty': '0.963086428',
  'avg_entry_price': '415.331363',
  'side': 'long',
  'market_value': '396.753085',
  'cost_basis': '399.999999',
  'unrealized_pl': '-3.246914',
  'unrealized_plpc': '-0.0081172850202932',
  'unrealized_intraday_pl': '-3.246914',
  'unrealized_intraday_plpc': '-0.0081172850202932',
  'current_price': '411.96',
  'lastday_price': '415.82',
  'change_today': '-0.0092828627771632',
  'qty_available': '0.963086428'}]

Check the open order

In [71]:
get_order_by_id("dcfbefb1-cbaa-4e60-9cef-9db801b8eb13", "e4d31b70-6ed8-4f61-8c2c-79d7ba769d94")

{'code': 40110000, 'message': 'request is not authorized'}


{'code': 40110000, 'message': 'request is not authorized'}

Query all orders, closed or open, for client:

In [72]:
get_list_of_orders(client_id, status="all")

NameError: name 'client_id' is not defined

Query open orders:

In [ ]:
get_list_of_orders(client_id, status="open")

[]

Cancel open order by id:

In [ ]:
cancel_open_order(client_id, order_created_response["id"])

Request error.


Cancel all open orders:

In [73]:
def cancel_all_open_orders(account_id):
    url = f"https://broker-api.sandbox.alpaca.markets/v1/trading/accounts/{account_id}/orders"

    response = requests.delete(url, headers=headers)

    if response.status_code == 207:
        succesful_cancellations = [order["id"] for order in response.json() if order["status"] == 200]
        unsuccesful_cancellations = [order["id"] for order in response.json() if order["status"] != 200]

        print(f"The following orders were cancelled: {', '.join(succesful_cancellations)}.")

        print(f"\nUnsuccesful cancellations: {', '.join(unsuccesful_cancellations)}.")

        return response.json()

Close an open position by id:

Open a position and close it after 7 days.
1. Open order. Register date.
2. Add to "positions_to_close" table.
3. Everyday, check "positions_to_close" table to check if there are any positions to close today. 
4. Issue position close order.
5. (Bonus) Monitor fullfilled price at market open and decide wether a limit price should be issued.

Monitor orders and positions. For the specific case of opening and closing positions everyday at closed market:
1. Have a table with order_id, order_date, close_date, status, intent_price, notional_amount, fulfilled_date, position_id, fulfilled_price, currency.
2. Place order. Register order_id, order_date, intent_price (that day's close price if market, or limit_price), notional_amount, status == pending.
3. Every day, check_status of non-definitive status (pending) orders and if open, then update to open and update fulfilled_date and fulfilled_price.
4. Every day, check planned_close_date == today() and status==open and close orders.
5. Start locally, but this table should be uploaded to MongoDB.
 

In [74]:
import sys
sys.path.append("..")

# CAMBIAR POR PARÁMETROS AWS

db_user = os.environ.get("MONGO_DB_USER")
db_pass = os.environ.get("MONGO_DB_PASS")
db_host = os.environ.get("MONGO_HOST")
database = os.environ.get("MONGO_DB_NAME")
mongodb_options = os.environ.get("MONGO_OPTIONS")

from src.support.data_load import MongoDBHandler

In [75]:
db_handle = MongoDBHandler(db_user=db_user, db_pass=db_pass, host=db_host, options=mongodb_options)

db_handle.connect_to_database(database)

Pinged your deployment. You successfully connected to MongoDB!


In [76]:
collection = db_handle.check_create_collection("trades")

Collection existent


### Encontrar información en MongoDB

De la colección, encontrar todo

In [122]:
collection.find()

# para convertir el cursor en información valiosa para nosotros podemos convertirlo a lista, como vemos a continuación
resultado_find_list = list(collection.find())
print(f"Hay {len(resultado_find_list)} órdenes en la colección:")
print(resultado_find_list[:2])

Hay 0 órdenes en la colección:
[]


Meter información de una orden

In [123]:
# order_id, order_date, planned_close_date, status, intent_price, notional_amount, fulfilled_date, position_id, fulfilled_price, currency.

mock_order = [{"content": {
    "alpaca_order_id": "order_12345",
    "order_date": "2025-01-18T20:00:00tz...",
    "alpaca_account_id": "account_12345",
    "planned_close_date": "2025-01-25T20:00:00tz...",
    "status": "PENDING",
    "intent_price": "10",
    "notional_amount": "100",
    "fulfilled_date": None,
    "position_id": None,
    "fulfilled_price": None,
    "currency": "USD"
}}]

In [124]:
db_handle.insert_documents(collection_name="trades", documents=mock_order)

InsertManyResult([ObjectId('67a6131126a7af951a0bf025')], acknowledged=True)

## Gestionar órdenes de la estrategia

1. **Crear orden**: Crearla en alpaca y registrarla en mongoDB.
2. **Actualizar estado de órdenes**: Comprobar estado en Alpaca y si es distinto actualizar en MongoDB.
3. **Cerrar órdenes**: Comprobar en MongoDB, cerrarlas en Alpaca, comprobar que el cierre ha sido exitoso y actualizar en MongoDB. Hacer la función de comprobación y cierre flexible y atómica en cuanto a input para poder reutilizarla a modo de demo sin que sea diario por la noche. 

In [125]:
order_created_response

{'id': 'ccfc806b-8340-4881-aed5-e76482ac2293',
 'client_order_id': 'ddd938b9-e18f-489e-a077-28195fea5c9c',
 'created_at': '2025-02-07T14:05:03.40040977Z',
 'updated_at': '2025-02-07T14:05:03.40284877Z',
 'submitted_at': '2025-02-07T14:05:03.40040977Z',
 'filled_at': None,
 'expired_at': None,
 'canceled_at': None,
 'failed_at': None,
 'replaced_at': None,
 'replaced_by': None,
 'replaces': None,
 'asset_id': 'b6d1aa75-5c9c-4353-a305-9e2caa1925ab',
 'symbol': 'MSFT',
 'asset_class': 'us_equity',
 'notional': '100',
 'qty': None,
 'filled_qty': '0',
 'filled_avg_price': None,
 'order_class': 'simple',
 'order_type': 'market',
 'type': 'market',
 'side': 'buy',
 'position_intent': 'buy_to_open',
 'time_in_force': 'day',
 'limit_price': None,
 'stop_price': None,
 'status': 'accepted',
 'extended_hours': False,
 'legs': None,
 'trail_percent': None,
 'trail_price': None,
 'hwm': None,
 'commission': '0',
 'commission_type': 'notional',
 'subtag': None,
 'source': None,
 'expires_at': '2025

1 - Crear y registrar orden

In [ ]:
from dateutil import parser

# Example string
date_string = '2025-01-23T12:10:06.272064904Z'

# Parse the string
parsed_date = parser.isoparse(date_string)
print(parsed_date)

2025-01-24 00:10:06.272064+00:00


In [81]:
def create_and_register_order(account_id: str,
                                symbol: str,
                                notional_amount: Union[str, float],
                                intent_open_price: float,
                                position_intent: str,
                                closure_date: datetime,
                                tp_sl: Dict,
                                # **argumentos_adicionales
                                ):
    
    order_response = create_order_for_client(account_id=account_id, symbol=symbol, notional=notional_amount, position_intent=position_intent) #, **argumentos_adicionales) # Could need additional parameters

    if order_response.get("id",None) == None:
        print(f"Order could not be placed due to {order_response.get('message', '_missing message_')}:")
        print(order_response)

    order_mongodb_record = [{"content": {
        "alpaca_order_id_open": order_response["id"],
        "alpaca_order_id_close": None,
        "TP_SL": tp_sl,
        "alpaca_account_id": account_id,
        "asset_id": order_response["asset_id"],
        "order_creation_date": parser.isoparse(order_response["created_at"]),
        "planned_close_date": closure_date,
        "planned_tp_sl_change_date": parser.isoparse(order_response["created_at"]) + pd.DateOffset(hours=12),
        "symbol": symbol,
        "position_intent": position_intent,
        "status_open": "PENDING",
        "status_close": None,
        "outcome": None,
        "intent_open_price": intent_open_price,
        "notional_amount": notional_amount,
        "qty": None,
        "fulfilled_open_date": None,
        "fulfilled_open_price": None,
        "fulfilled_close_date": None,
        "fulfilled_close_price": None,
        "pct_profit_trade": None,
        "currency": "USD"
    }}]

    insertion_result = db_handle.insert_documents(collection_name="trades", documents=order_mongodb_record)

    if insertion_result.inserted_ids:
        print("Document inserted successfully with _id:", insertion_result.inserted_ids)
    else:
        print("Insertion failed.")
    
    return insertion_result.inserted_ids, order_mongodb_record[0]

In [128]:
clients_account_id

{'client_1': 'dcfbefb1-cbaa-4e60-9cef-9db801b8eb13',
 'client_2': 'edafa6db-b501-4d69-95c8-b92793a3680c',
 'client_3': 'b4363824-8773-4f6f-96d3-e24c1e80e208'}

- NOTA IMPORTANTE: habría que ajustar bien la asignación de fecha-hora de cierre. Lo ideal para este caso es cuando el precio no se mueva. Tener función de comprobación del cierre de mercado (a lo mejor es un detalle de next steps.)

In [87]:
closure_date=(datetime.today() + timedelta(days=7)).replace(tzinfo=timezone.utc)
closure_date.date()

parser.isoparse(order_response["created_at"])

NameError: name 'order_response' is not defined

In [83]:
closure_date

datetime.datetime(2025, 2, 14, 17, 26, 32, 931950, tzinfo=datetime.timezone.utc)

In [130]:
inserted_id, order_mongodb_record = create_and_register_order(clients_account_id["client_1"], 
                                                              symbol="AAPL", 
                                                              notional_amount="300", 
                                                              intent_open_price="223.83",
                                                              position_intent="buy_to_open",
                                                              closure_date=closure_date)

Succesfully placed order 2b2092aa-c2db-4d57-802d-216ff95570de for buy_to_open of the symbol AAPL. See details in response.
Document inserted successfully with _id: [ObjectId('67a6131226a7af951a0bf026')]


### Update order status and info in MongoDB

1.1 - Consultar orden por id de Alpaca

In [49]:
# These 2 are quivalent:
# first mode
collection.find_one({'content.alpaca_order_id_open': order_mongodb_record["content"]["alpaca_order_id_open"]})

# explicit mode
collection.find_one({'content.alpaca_order_id_open': {"$eq":order_mongodb_record["content"]["alpaca_order_id_open"]}})

# # By MongoDB objectID it would be
# list(collection.find({'_id':ObjectId(inserted_id)}))

NameError: name 'collection' is not defined

2 - Actualizar estado de la orden


In [132]:
def update_order_field(direction:str, # this specifices which alpaca_order_id to use for update
                       alpaca_order_id: str, field_to_update: str, new_field_value: str):
    # update status
    db_handle.db["trades"].update_one(
        {f'content.alpaca_order_id_{direction}': alpaca_order_id}, # Filter
        {"$set": {f'content.{field_to_update}': new_field_value}}  # Update
    )


2.1 - Chequear orden en mongo: Lista de órdenes en PENDING

In [133]:
pending_orders = list(collection.find({'content.status_open':'PENDING'}))
pending_orders

[{'_id': ObjectId('67a6131226a7af951a0bf026'),
  'content': {'alpaca_order_id_open': '2b2092aa-c2db-4d57-802d-216ff95570de',
   'alpaca_order_id_close': None,
   'alpaca_account_id': 'dcfbefb1-cbaa-4e60-9cef-9db801b8eb13',
   'asset_id': 'b0b6dd9d-8b9b-48a9-ba46-b9d54906e415',
   'order_creation_date': datetime.datetime(2025, 2, 7, 14, 5, 6, 575000),
   'planned_close_date': datetime.datetime(2025, 2, 14, 15, 5, 5, 955000),
   'symbol': 'AAPL',
   'position_intent': 'buy_to_open',
   'status_open': 'PENDING',
   'status_close': None,
   'intent_open_price': '223.83',
   'notional_amount': '300',
   'qty': None,
   'fulfilled_open_date': None,
   'fulfilled_open_price': None,
   'fulfilled_close_date': None,
   'fulfilled_close_price': None,
   'pct_profit_trade': None,
   'currency': 'USD'}}]

2.2 - Actualizar ordenes pending por id

In [134]:
def refresh_pending_status_open(account_id, alpaca_order_id):

    # get order info from alpaca
    order_in_alpaca = get_order_by_id(account_id, alpaca_order_id)
    
    # update status field
    update_order_field(order_in_alpaca["id"], "status_open", order_in_alpaca["status"])

    # if filled, update price, date and quantity
    if order_in_alpaca["status"] == "filled":
        # update filled_price
        update_order_field(direction="open", 
                            alpaca_order_id=order_in_alpaca["id"], 
                            field_to_update="fulfilled_open_price", 
                            new_field_value=order_in_alpaca["filled_avg_price"])
        # update filled_date
        update_order_field(direction="open", 
                            alpaca_order_id=order_in_alpaca["id"], 
                            field_to_update="fulfilled_open_date", 
                            new_field_value=order_in_alpaca["filled_at"])
        # update qty
        update_order_field(direction="open", 
                            alpaca_order_id=order_in_alpaca["id"], 
                            field_to_update="qty", 
                            new_field_value=order_in_alpaca["filled_qty"])

In [79]:
closure_date

datetime.datetime(2025, 2, 14, 17, 26, 32, 931950, tzinfo=datetime.timezone.utc)

In [90]:
parser.isoparse("2024-01-01")

datetime.datetime(2024, 1, 1, 0, 0)

In [116]:
tp_n_sl_1 = closure_date + pd.DateOffset(hours=12)
tp_n_sl_2 = closure_date + pd.DateOffset(hours=36)
tp_n_sl_3 = closure_date + pd.DateOffset(hours=60)
tp_n_sl_4 = closure_date + pd.DateOffset(hours=84)
tp_n_sl_5 = closure_date + pd.DateOffset(hours=108)
date_format = "%Y-%m-%dT%H:%M:%S"


inserted_id, order_mongodb_record = create_and_register_order(clients_account_id["client_1"], 
                                                              symbol="AAPL", 
                                                              notional_amount="1", 
                                                              intent_open_price="223.83",
                                                              position_intent="buy_to_open",
                                                              closure_date=closure_date,
                                                              tp_sl= {
                                                                  "TP": {tp_n_sl_1.strftime(date_format): ("223.83", None),
                                                                         tp_n_sl_2.strftime(date_format): ("224.83", None),
                                                                         tp_n_sl_3.strftime(date_format): ("225.83", None),
                                                                         tp_n_sl_4.strftime(date_format): ("226.83", None),
                                                                         tp_n_sl_5.strftime(date_format): ("227.83", None)},
                                                                  "SL": {tp_n_sl_1.strftime(date_format): ("222.83", None),
                                                                         tp_n_sl_2.strftime(date_format): ("221.83", None),
                                                                         tp_n_sl_3.strftime(date_format): ("220.83", None),
                                                                         tp_n_sl_4.strftime(date_format): ("219.83", None),
                                                                         tp_n_sl_5.strftime(date_format): ("218.83", None)}
                                                              })

Succesfully placed order 59cc8618-f204-4c10-b18f-769bebcd4030 for buy_to_open of the symbol AAPL. See details in response.
Document inserted successfully with _id: [ObjectId('67a63d8a1251622c825de7e6')]


Now, lets go through the workflow to post the first TP and the subsequent.

In [118]:
order_mongodb_record = db_handle.db["trades"].find_one({'content.alpaca_order_id_open': {"$eq":"59cc8618-f204-4c10-b18f-769bebcd4030"}}) 
order_mongodb_record

{'_id': ObjectId('67a63d8a1251622c825de7e6'),
 'content': {'alpaca_order_id_open': '59cc8618-f204-4c10-b18f-769bebcd4030',
  'alpaca_order_id_close': None,
  'TP_SL': {'TP': {'2025-02-15T05:37:19': ['223.83', None],
    '2025-02-16T05:37:19': ['224.83', None],
    '2025-02-17T05:37:19': ['225.83', None],
    '2025-02-18T05:37:19': ['226.83', None],
    '2025-02-19T05:37:19': ['227.83', None]},
   'SL': {'2025-02-15T05:37:19': ['222.83', None],
    '2025-02-16T05:37:19': ['221.83', None],
    '2025-02-17T05:37:19': ['220.83', None],
    '2025-02-18T05:37:19': ['219.83', None],
    '2025-02-19T05:37:19': ['218.83', None]}},
  'alpaca_account_id': 'b5284860-b133-458b-8140-8fb5e59d8439',
  'asset_id': 'b0b6dd9d-8b9b-48a9-ba46-b9d54906e415',
  'order_creation_date': datetime.datetime(2025, 2, 7, 17, 6, 18, 142000),
  'planned_close_date': datetime.datetime(2025, 2, 14, 17, 37, 19, 967000),
  'planned_tp_sl_change_date': datetime.datetime(2025, 2, 8, 5, 6, 18, 142000),
  'symbol': 'AAPL',
  

In [102]:
parser.isoparse("2024-01-01")

datetime.datetime(2024, 1, 1, 0, 0)

In [ ]:
# [(parser.isoparse(date), tp) for date, tp in order_mongodb_record["content"]["TP_SL"]["TP"].items() if parser.isoparse(date) <= datetime.now()]

datetime_now = datetime(2025,2,16)
date_price_tuple_list_tp = [(parser.isoparse(date), tp) for date, tp in order_mongodb_record["content"]["TP_SL"]["TP"].items() if parser.isoparse(date) <= datetime_now]

(datetime.datetime(2025, 2, 15, 5, 37, 19), '223.83')

In [ ]:
def refresh_pending_status(direction:str,account_id, alpaca_order_id):

    # get order info from alpaca
    order_in_alpaca = get_order_by_id(account_id, alpaca_order_id)
    
    # update status field
    update_order_field(direction=direction,
                       alpaca_order_id=order_in_alpaca["id"], 
                       field_to_update=f"status_{direction}", 
                       new_field_value=order_in_alpaca["status"])

    # if filled, update price, date and quantity
    if order_in_alpaca["status"] == "filled":
        # update filled_price
        update_order_field(direction = direction, 
                            alpaca_order_id = order_in_alpaca["id"], 
                            field_to_update = f"fulfilled_{direction}_price", 
                            new_field_value = order_in_alpaca["filled_avg_price"])
        # update filled_date
        update_order_field(direction = direction, 
                            alpaca_order_id = order_in_alpaca["id"], 
                            field_to_update = f"fulfilled_{direction}_date", 
                            new_field_value = order_in_alpaca["filled_at"])
        
        if direction == "open":
            # update qty
            update_order_field(direction = direction, 
                                alpaca_order_id = order_in_alpaca["id"], 
                                field_to_update ="qty", 
                                new_field_value = order_in_alpaca["filled_qty"])
            
            # set_tp_n_sl()

        else: # close
            # update pct_profit
            filled_open_price = float(db_handle.db["trades"].find_one({f'content.alpaca_order_id_close': alpaca_order_id})["content"]["fulfilled_open_price"])
            filled_closed_price = float(order_in_alpaca["filled_avg_price"])
            pct_profit_trade = filled_closed_price / filled_open_price - 1
            
            update_order_field(direction = direction, 
                                alpaca_order_id = order_in_alpaca["id"], 
                                field_to_update = "pct_profit_trade", 
                                new_field_value = pct_profit_trade)

2.3 Pasar ids de funciones pending a la función de actualizar status pending

In [136]:
def refresh_pending_statuses_open(account_id):
    
    pending_orders_mongodb = list(collection.find({'content.status_open':'PENDING'}))

    for order in pending_orders_mongodb:
        order = order["content"]
        refresh_pending_status_open(account_id, order["alpaca_order_id_open"])

In [137]:
def refresh_statuses(direction:str, account_id):
    
    pending_orders_mongodb = list(collection.find({"$or": 
                                                   [{f'content.status_{direction}':'PENDING'},
                                                    {f'content.status_{direction}':'accepted'}]}))

    for order in pending_orders_mongodb:
        order = order["content"]
        refresh_pending_status(direction=direction, account_id=account_id, alpaca_order_id=order[f"alpaca_order_id_{direction}"])

Comprobando:

In [138]:
refresh_statuses(direction="open", account_id=clients_account_id["client_1"])

{'id': '2b2092aa-c2db-4d57-802d-216ff95570de', 'client_order_id': '65b4583d-293f-42ce-bb33-9eeb75b0f110', 'created_at': '2025-02-07T14:05:06.575218435Z', 'updated_at': '2025-02-07T14:05:06.576262185Z', 'submitted_at': '2025-02-07T14:05:06.575218435Z', 'filled_at': None, 'expired_at': None, 'canceled_at': None, 'failed_at': None, 'replaced_at': None, 'replaced_by': None, 'replaces': None, 'asset_id': 'b0b6dd9d-8b9b-48a9-ba46-b9d54906e415', 'symbol': 'AAPL', 'asset_class': 'us_equity', 'notional': '300', 'qty': None, 'filled_qty': '0', 'filled_avg_price': None, 'order_class': 'simple', 'order_type': 'market', 'type': 'market', 'side': 'buy', 'position_intent': 'buy_to_open', 'time_in_force': 'day', 'limit_price': None, 'stop_price': None, 'status': 'accepted', 'extended_hours': False, 'legs': None, 'trail_percent': None, 'trail_price': None, 'hwm': None, 'commission': '0', 'commission_type': 'notional', 'subtag': None, 'source': None, 'expires_at': '2025-02-07T21:00:00Z'}


In [139]:
# refresh_pending_statuses_open(clients_account_id["client_1"])

3. **Cerrar órdenes**: Comprobar en MongoDB, cerrarlas en Alpaca, comprobar que el cierre ha sido exitoso y actualizar en MongoDB. Hacer la función de comprobación y cierre flexible y atómica en cuanto a input para poder reutilizarla a modo de demo sin que sea diario por la noche. 

1 - Comprobar en Mongo qué órdenes o posiciones se deben cerrar (habría que ajustar bien la asignación de fecha-hora de cierre)

In [ ]:
closing_datetime = datetime(2025, 1, 30)
# Comprobar qué órdenes coinciden ya han pasado esa fecha y siguen abiertas
def check_open_due_orders(closing_datetime: datetime):
    due_order_list_mongodb = list(db_handle.db["trades"].find(
                {"$and":
                [{'content.planned_close_date': {"$lte": closing_datetime}}, # After closing time filter
                {'content.status_open': 'filled'}
                ]}))
    
    due_order_list_mongodb = [due_order["content"] for due_order in due_order_list_mongodb]
    
    return due_order_list_mongodb

def check_open_due_orders_today():
    due_order_list_mongodb = check_open_due_orders(closing_datetime = datetime.today())

    return due_order_list_mongodb

2.1 - Update status closed in database and close qty of position equal to quantity of closing order

In [141]:
datetime.today().date()

datetime.date(2025, 2, 7)

In [147]:
mongodb_order_dict = db_handle.db["trades"].find_one({"content.alpaca_order_id_open": "2b2092aa-c2db-4d57-802d-216ff95570de"})
mongodb_order_dict = mongodb_order_dict["content"]

In [144]:
# response_close = close_symbol_position_client(account_id=mongodb_order_dict["alpaca_account_id"],
#                                  symbol=mongodb_order_dict["symbol"], 
#                                  magnitude=mongodb_order_dict["qty"], 
#                                  mode = "qty")

In [145]:
get_order_by_id(mongodb_order_dict["alpaca_account_id"],"1bba1637-3e98-4846-9090-ef556ac6e571")

TypeError: 'NoneType' object is not subscriptable

In [ ]:
def close_due_order_position(mongodb_order_dict: dict):
    # close qty amount in corresponding asset position
    response_close = close_symbol_position_client(account_id=mongodb_order_dict["alpaca_account_id"],
                                 symbol=mongodb_order_dict["symbol"], 
                                 magnitude=mongodb_order_dict["qty"], 
                                 mode = "qty")
    
    # initialize close order status
    update_order_field(direction="open", # IMPORTANT: direction="open" to use alpaca_order_id_open
                       alpaca_order_id = mongodb_order_dict["alpaca_order_id_open"],
                        field_to_update = "status_close", 
                        new_field_value = "PENDING")
    
    # update alpaca_order_id_close
    update_order_field(direction="open", # IMPORTANT: direction="open" to use alpaca_order_id_open
                       alpaca_order_id = mongodb_order_dict["alpaca_order_id_open"],
                        field_to_update = "alpaca_order_id_close", 
                        new_field_value = response_close["id"])
    
    return response_close

def cancel_open_order_mongodb(account_id, alpaca_order_id):    
    cancel_open_order(account_id = account_id,
                      order_id = alpaca_order_id)
    
    # initialize close order status
    update_order_field(direction="open", # IMPORTANT: direction="open" to use alpaca_order_id_open
                       alpaca_order_id = alpaca_order_id,
                        field_to_update = "status_open", 
                        new_field_value = "cancelled")
    

    
    

In [ ]:
cancel_open_order_mongodb("dcfbefb1-cbaa-4e60-9cef-9db801b8eb13", "52477caf-d80a-4846-b5ee-ce7337637962")

Order '52477caf-d80a-4846-b5ee-ce7337637962' cancelled succesfully.


{'id': '1bba1637-3e98-4846-9090-ef556ac6e571', 'client_order_id': '13e948c8-6f89-410f-a845-bfced31ef990', 'created_at': '2025-01-24T07:56:30.687347177Z', 'updated_at': '2025-01-24T07:56:30.687772617Z', 'submitted_at': '2025-01-24T07:56:30.687347177Z', 'filled_at': None, 'expired_at': None, 'canceled_at': None, 'failed_at': None, 'replaced_at': None, 'replaced_by': None, 'replaces': None, 'asset_id': 'b0b6dd9d-8b9b-48a9-ba46-b9d54906e415', 'symbol': 'AAPL', 'asset_class': 'us_equity', 'notional': None, 'qty': '1.338473069', 'filled_qty': '0', 'filled_avg_price': None, 'order_class': '', 'order_type': 'market', 'type': 'market', 'side': 'sell', 'position_intent': 'sell_to_close', 'time_in_force': 'day', 'limit_price': None, 'stop_price': None, 'status': 'accepted', 'extended_hours': False, 'legs': None, 'trail_percent': None, 'trail_price': None, 'hwm': None, 'subtag': None, 'source': None, 'expires_at': '2025-01-24T21:00:00Z'}


2.2 - Close order

In [ ]:
response_close = close_due_order_position(mongodb_order_dict)

{'id': '1bba1637-3e98-4846-9090-ef556ac6e571', 'client_order_id': '13e948c8-6f89-410f-a845-bfced31ef990', 'created_at': '2025-01-24T07:56:30.687347177Z', 'updated_at': '2025-01-24T07:56:30.687772617Z', 'submitted_at': '2025-01-24T07:56:30.687347177Z', 'filled_at': None, 'expired_at': None, 'canceled_at': None, 'failed_at': None, 'replaced_at': None, 'replaced_by': None, 'replaces': None, 'asset_id': 'b0b6dd9d-8b9b-48a9-ba46-b9d54906e415', 'symbol': 'AAPL', 'asset_class': 'us_equity', 'notional': None, 'qty': '1.338473069', 'filled_qty': '0', 'filled_avg_price': None, 'order_class': '', 'order_type': 'market', 'type': 'market', 'side': 'sell', 'position_intent': 'sell_to_close', 'time_in_force': 'day', 'limit_price': None, 'stop_price': None, 'status': 'accepted', 'extended_hours': False, 'legs': None, 'trail_percent': None, 'trail_price': None, 'hwm': None, 'subtag': None, 'source': None, 'expires_at': '2025-01-24T21:00:00Z'}


2.3 - Refresh the state to the database

In [ ]:
refresh_statuses(direction="close", account_id=clients_account_id["client_1"])

### Create TP/SL and update them

If a trade hits TP, give it outcome "TP". IF hits "SL", outcome "SL". Else "close".

Opening a trade, will have as many TP and SL as upper and lower bounds will the prediction. Once order is filled, set up orders to close the position at

In [ ]:
def check_open_due_orders(closing_datetime: datetime):
    due_order_list_mongodb = list(db_handle.db["trades"].find(
                {"$and":
                [{'content.planned_close_date': {"$lte": closing_datetime}}, # After closing time filter
                {'content.status_open': 'filled'}
                ]}))
    
    due_order_list_mongodb = [due_order["content"] for due_order in due_order_list_mongodb]
    
    return due_order_list_mongodb

In [ ]:
from pymongo import MongoClient

# Connect to MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['your_database']
collection = db['your_collection']

# Define the filter to find the document to update
filter = {'_id': 12345}  # Use the document's identifier or any filter

# Define the update operation
update = {
    '$set': {'field_name': 'new_value'}  # Specify the fields to update
}

# Update the document
result = collection.update_one(filter, update)

# Check the result
print(f'Matched: {result.matched_count}, Modified: {result.modified_count}')


In [ ]:
def get_tp_sl_date_price_list(tp_sl_dict, tp_sl_choice:str, qty: float, datetime_now: datetime = datetime.now()):
    if not tp_sl_choice in ["TP","SL"]:
        raise ValueError("Type of TP/SL not implemented.")
    
    date_price_tuple_list_tp = [(parser.isoparse(date), price) for date, (price, order_id) in tp_sl_dict[tp_sl_choice].items() if parser.isoparse(date) <= datetime_now]

    latest_price = date_price_tuple_list_tp[0]

    inserted_alpaca_id, _ = create_order_for_client(account_id=clients_account_id["client_1"], symbol="MSFT", qty="0.05", side="sell",type="limit", limit_price=latest_price)
    
    return inserted_alpaca_id, 

In [ ]:
def set_tp_n_sl(direction, alpaca_order_id_open, datetime_now: datetime = datetime.now()):

    # Get mongo db record info of trade
    order_mongodb_record = db_handle.db["trades"].find_one({'content.alpaca_order_id_open': {"$eq":alpaca_order_id_open}}) 

    tp_sl_dict = order_mongodb_record["content"]["TP_SL"]
    
    # Create SL
    stop_loss_list = get_tp_sl_date_price_list(tp_sl_dict, tp_sl_choice="SL", datetime_now=datetime_now)
    stop_loss_price = stop_loss_list[0]
    create_order_for_client(account_id=clients_account_id["client_1"], symbol="MSFT", qty="0.05", side="sell",type="limit", limit_price=stop_loss_price)

    # Create TP
    stop_loss_list = get_tp_sl_date_price_list(tp_sl_dict, tp_sl_choice="SL", datetime_now=datetime_now)
    stop_loss_price = stop_loss_list[0]
    create_order_for_client(account_id=clients_account_id["client_1"], symbol="MSFT", qty="0.05", side="sell",type="limit", limit_price=stop_loss_price)


    update_order_field(direction = direction, 
                            alpaca_order_id = alpaca_order_id_open, 
                            field_to_update = "planned_tp_sl_change_date", 
                            new_field_value = order_mongodb_record["planned_tp_sl_change_date"] + pd.DateOffset(hours=24))

## Workflow

Sabado por la noche se generan predicciones.

#### A la apertura de los mercados se consulta en MongoDB los stocks que tienen predicciones para los mercados abiertos. 
#### At market open, stocks that operate in those markets are queried to check their predictions 

In [119]:
# First, have market opening dict for tickers

# Second, have a cron and .sh file programmed for those hours

# Third, check those tickers open requirements
db_handle = MongoDBHandler(db_user=db_user, db_pass=db_pass, host=db_host, options=mongodb_options)

db_handle.connect_to_database(database)

collection = db_handle.db["stocks_predictions"]

result_find_list = list(collection.find())
print(f"There {len(result_find_list)} trades predicted in the collection:")
print(result_find_list[:2])


Pinged your deployment. You successfully connected to MongoDB!
There 32 trades predicted in the collection:
[{'_id': ObjectId('67a5f8478c19b8c7ba9eabfa'), 'symbol': 'AAPL', 'forecast_horizon_start': datetime.datetime(2025, 2, 3, 0, 0), 'mean_upper_bound': 241.1686881567026, 'mean_lower_bound': 231.00618293187918, 'open_price_for_rrr': 235.62550348861706, 'adj_open_price_for_rrr': 235.15519310241223, 'mean_pot_profit_pct': 0.003964371058871988, 'forecast_creation': '2025-02-07', 'target_risk_reward': 2.5, 'comission_rate': 0.002, 'bounds': {'2025-02-03': {'forecast_day': 1, 'upper_bound': 238.39220277715108, 'lower_bound': 233.58717790450936}, '2025-02-04': {'forecast_day': 2, 'upper_bound': 239.8073237985267, 'lower_bound': 231.89004029777902}, '2025-02-05': {'forecast_day': 3, 'upper_bound': 241.54761136439447, 'lower_bound': 230.96673499501046}, '2025-02-06': {'forecast_day': 4, 'upper_bound': 242.4898937161684, 'lower_bound': 229.80083659916093}, '2025-02-07': {'forecast_day': 5, 'u

Find those with dates predicted for tomorrow, taking the example of AAPL and NVDA:

In [83]:
tickers_list = ["AAPL","NVDA"]
date_simulation = datetime(2025, 2, 3)
tickers_today = list(collection.find({"$and": [{"forecast_horizon_start": date_simulation},{"symbol": {"$in": tickers_list}}]}))
tickers_today

[{'_id': ObjectId('67a5f8478c19b8c7ba9eabfa'),
  'symbol': 'AAPL',
  'forecast_horizon_start': datetime.datetime(2025, 2, 3, 0, 0),
  'mean_upper_bound': 241.1686881567026,
  'mean_lower_bound': 231.00618293187918,
  'open_price_for_rrr': 235.62550348861706,
  'adj_open_price_for_rrr': 235.15519310241223,
  'mean_pot_profit_pct': 0.003964371058871988,
  'forecast_creation': '2025-02-07',
  'target_risk_reward': 2.5,
  'comission_rate': 0.002,
  'bounds': {'2025-02-03': {'forecast_day': 1,
    'upper_bound': 238.39220277715108,
    'lower_bound': 233.58717790450936},
   '2025-02-04': {'forecast_day': 2,
    'upper_bound': 239.8073237985267,
    'lower_bound': 231.89004029777902},
   '2025-02-05': {'forecast_day': 3,
    'upper_bound': 241.54761136439447,
    'lower_bound': 230.96673499501046},
   '2025-02-06': {'forecast_day': 4,
    'upper_bound': 242.4898937161684,
    'lower_bound': 229.80083659916093},
   '2025-02-07': {'forecast_day': 5,
    'upper_bound': 243.60640912727246,
    '

Build the conditions dataframe for trade opening:

In [86]:
def get_evaluation_trade_open(mongo_collection: object, symbols_list: List[str], date: datetime = datetime.today()):
    tickers_list = list(mongo_collection.find({"$and": [{"forecast_horizon_start": date},{"symbol": {"$in": symbols_list}}]}))

    trade_evaluation_open_dict = {"symbol": [],
                                  "adj_open_price_for_rrr": [],
                                  "upper_bound_TP": [],
                                  "lower_bound_SL": []}
    
    
    for ticker in tickers_list:
        trade_evaluation_open_dict["symbol"].append(ticker["symbol"])
        trade_evaluation_open_dict["adj_open_price_for_rrr"].append(ticker["adj_open_price_for_rrr"])
        trade_evaluation_open_dict["upper_bound_TP"].append(ticker["bounds"][date.strftime("%Y-%m-%d")]["upper_bound"])
        trade_evaluation_open_dict["lower_bound_SL"].append(ticker["bounds"][date.strftime("%Y-%m-%d")]["lower_bound"])


    return pd.DataFrame(trade_evaluation_open_dict)

tickers_required_price = get_evaluation_trade_open(collection, symbols_list=tickers_list, date=date_simulation)
tickers_required_price

,symbol,adj_open_price_for_rrr,upper_bound_TP,lower_bound_SL
0,AAPL,235.155193,238.392203,233.587178
1,NVDA,121.627657,123.509440,118.285658


#### If its current price is below the required, open trade:
- Note: With a limit set up by the exposure and other conditions. I am going to open 10 for this PoC

In [93]:
ALPHA_VANTAGE_API_KEY = os.environ.get("ALPHA_VANTAGE_API_KEY")

def get_last_ticker_value(ticker_symbol: str, api_key: str):
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={ticker_symbol}&interval=5min&apikey={api_key}'
    r = requests.get(url)
    data = r.json()
    last_close = float( next( iter( data['Time Series (5min)'].values() ) )['4. close'] )

    return last_close

tickers_required_price["last_price"] = tickers_required_price["symbol"].apply(lambda symbol: get_last_ticker_value(symbol, ALPHA_VANTAGE_API_KEY))
tickers_required_price["open_trade"] = np.where(tickers_required_price["last_price"] < tickers_required_price["adj_open_price_for_rrr"], 1 ,0)
tickers_required_price

,symbol,adj_open_price_for_rrr,upper_bound_TP,lower_bound_SL,last_price,open_trade
0,AAPL,235.155193,238.392203,233.587178,233.03,1
1,NVDA,121.627657,123.509440,118.285658,128.14,0


In [ ]:
create_and_register_order()